In [3]:
import pandas as pd
import pyhdfs

In [4]:
client = pyhdfs.HdfsClient(hosts="10.120.26.200,50070",user_name="spark")

In [5]:
select_stock = "1101.TW.csv"
stock_loc = '/user/spark/spark101/spark203/'

In [6]:

with client.open(f'{stock_loc}{select_stock}') as inputfile:
    df =  pd.read_csv(inputfile,index_col ='Date')

In [37]:
df.columns = map(str.lower, df.columns)
df = df.dropna().astype(float)
df = df[df['volume'] != 0]

In [38]:
import talib
from talib import abstract
# import yfinance as yf

In [39]:
print(abstract.STOCH)

STOCH([input_arrays], [fastk_period=5], [slowk_period=3], [slowk_matype=0], [slowd_period=3], [slowd_matype=0])

Stochastic (Momentum Indicators)

Inputs:
    prices: ['high', 'low', 'close']
Parameters:
    fastk_period: 5
    slowk_period: 3
    slowk_matype: 0
    slowd_period: 3
    slowd_matype: 0
Outputs:
    slowk
    slowd


In [40]:
kd = abstract.STOCH(df, fastk_period = 9)
kd

,slowk,slowd
Date,,
2000-01-04,NaN,NaN
2000-01-05,NaN,NaN
2000-01-06,NaN,NaN
2000-01-07,NaN,NaN
2000-01-10,NaN,NaN
...,...,...
2020-08-04,64.896620,63.157775
2020-08-05,77.195691,67.902025
2020-08-06,92.433767,78.175359


In [41]:
print(abstract.MACD)

MACD([input_arrays], [fastperiod=12], [slowperiod=26], [signalperiod=9])

Moving Average Convergence/Divergence (Momentum Indicators)

Inputs:
    price: (any ndarray)
Parameters:
    fastperiod: 12
    slowperiod: 26
    signalperiod: 9
Outputs:
    macd
    macdsignal
    macdhist


In [42]:
macd = abstract.MACD(df)
type(macd)

pandas.core.frame.DataFrame

In [43]:
print(abstract.WILLR)

WILLR([input_arrays], [timeperiod=14])

Williams' %R (Momentum Indicators)

Inputs:
    prices: ['high', 'low', 'close']
Parameters:
    timeperiod: 14
Outputs:
    real


In [44]:
willr = abstract.WILLR(df)
willr = willr.to_frame()
willr.columns=['willr']
willr

,willr
Date,
2000-01-04,NaN
2000-01-05,NaN
2000-01-06,NaN
2000-01-07,NaN
2000-01-10,NaN
...,...
2020-08-04,-22.222068
2020-08-05,-7.547202
2020-08-06,-0.000000


In [45]:
print(abstract.BBANDS)

BBANDS([input_arrays], [timeperiod=5], [nbdevup=2], [nbdevdn=2], [matype=0])

Bollinger Bands (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 5
    nbdevup: 2
    nbdevdn: 2
    matype: 0 (Simple Moving Average)
Outputs:
    upperband
    middleband
    lowerband


In [46]:
bband = abstract.BBANDS(df,timeperiod = 20)
type(bband)

pandas.core.frame.DataFrame

In [47]:
pdList =[df, kd, macd, willr, bband]
summary_df = pd.concat(pdList, join='outer',axis=1)
summary_df

,open,high,low,close,adj close,volume,slowk,slowd,macd,macdsignal,macdhist,willr,upperband,middleband,lowerband
Date,,,,,,,,,,,,,,,
2000-01-04,17.805401,18.226801,17.805401,18.226801,7.236699,9.716852e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,17.858000,19.174999,17.858000,18.700899,7.424934,1.499255e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,18.542900,18.753599,17.752701,17.910700,7.111196,7.448073e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,17.173201,17.963400,16.857100,17.858000,7.090272,8.759947e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-10,17.910700,18.542900,17.647301,18.437500,7.320355,7.871879e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-04,44.950001,45.650002,44.950001,45.400002,45.400002,2.088774e+07,64.896620,63.157775,0.346311,0.413200,-0.066889,-22.222068,45.787382,45.1200,44.452617
2020-08-05,45.599998,46.650002,45.599998,46.450001,46.450001,4.054277e+07,77.195691,67.902025,0.432534,0.417067,0.015468,-7.547202,46.069364,45.1850,44.300635
2020-08-06,46.950001,47.150002,46.500000,47.150002,47.150002,3.407786e+07,92.433767,78.175359,0.551000,0.443853,0.107147,-0.000000,46.509359,45.2775,44.045641


In [48]:
# sma_5 = abstract.SMA(df,timeperiod=5)
# ema_5 = abstract.EMA(df,timeperiod=5)
# wma_5 = abstract.WMA(df,timeperiod=5)


In [49]:
# sma_10 = abstract.SMA(df,timeperiod=10)
# ema_10 = abstract.EMA(df,timeperiod=10)
# wma_10 = abstract.WMA(df,timeperiod=10)

In [50]:
# sma_25 = abstract.SMA(df,timeperiod=25)
# ema_25 = abstract.EMA(df,timeperiod=25)
# wma_25 = abstract.WMA(df,timeperiod=25)

In [51]:
# sma_50 = abstract.SMA(df,timeperiod=50)
# ema_50 = abstract.EMA(df,timeperiod=50)
# wma_50 = abstract.WMA(df,timeperiod=50)

In [52]:
# sma_60 = abstract.SMA(df,timeperiod=60)
# ema_60 = abstract.EMA(df,timeperiod=60)
# wma_60 = abstract.WMA(df,timeperiod=60)

In [53]:
# sma_300 = abstract.SMA(df,timeperiod=300)
# ema_300 = abstract.EMA(df,timeperiod=300)
# wma_300 = abstract.WMA(df,timeperiod=300)

In [54]:
def ma_by_day(days,df):
    ma_df = pd.DataFrame()
    sma = abstract.SMA(df,timeperiod=days)
    ema = abstract.EMA(df,timeperiod=days)
    wma = abstract.WMA(df,timeperiod=days)
    ma_columns = [f'sma_{days}',f'ema_{days}',f'wma_{days}']
    for index, x in enumerate([sma,ema,wma]):
        x = x.to_frame()
        x.columns=[ma_columns[index]]
        ma_df = pd.concat([ma_df,x],join='outer',axis=1)
    return ma_df

In [55]:
print(ma_by_day(5,df))

sma_5      ema_5      wma_5
Date                                       
2000-01-04        NaN        NaN        NaN
2000-01-05        NaN        NaN        NaN
2000-01-06        NaN        NaN        NaN
2000-01-07        NaN        NaN        NaN
2000-01-10  18.226780  18.226780  18.198680
...               ...        ...        ...
2020-08-04  45.150000  45.141546  45.170001
2020-08-05  45.430000  45.577698  45.603334
2020-08-06  45.790001  46.101799  46.176668
2020-08-07  46.210001  46.434532  46.613334
2020-08-10  46.550001  46.506355  46.760001

[5044 rows x 3 columns]


In [56]:
ma_days = [5,10,25,50,60,300]

In [57]:
for i in ma_days:
   summary_df = pd.concat([summary_df, ma_by_day(i,df)], join='outer',axis=1)


In [58]:
summary_df = summary_df.fillna(0)

In [60]:
summary_df.to_csv('./selected_stock.csv',encoding='utf-8-sig')

In [35]:
client.copy_from_local('./selected_stock.csv','/user/spark/stock/FC/tmp/selected_stock.csv')
